In [1]:
%load_ext kedro

[10/16/25 20:29:30] INFO     Using                                                                  ]8;id=921977;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=607461;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.1                
                             1/site-packages/kedro/framework/project/rich_logging.yml' as logging                  
                             configuration.                                                                        

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=269474;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=274709;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=375493;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=33568;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=178176;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=644901;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#180\180]8;;\
                             /home/zishan/Documents/projects/commodity-price-forecasting.                          
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/16/25 20:29:30] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=633919;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=210590;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/16/25 20:29:32] INFO     Kedro project commodity_price_forecasting                              ]8;id=606397;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=925195;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=915854;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=903297;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#147\147]8;;\
                             'pipelines'                                                                           

In [ ]:
df = catalog.load('clean_data')
df.info()

[10/16/25 20:29:55] INFO     Loading data from clean_data (ParquetDataset)...                  ]8;id=131020;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=143161;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

In [57]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [55]:
def get_seasonal_stats(item_name):
    item_data = df[df['item'] == item_name].copy().sort_values('date')
    item_data.set_index('date', inplace=True)
    price_series = item_data['high_price']
    result = seasonal_decompose(price_series, model='additive', period=7)
    return {
        'item': item_name,
        'trend_strength': result.trend.std(),
        'seasonality_strength': result.seasonal.std(),
        'residual_strength': result.resid.std()
    }

stats_list = [get_seasonal_stats(item) for item in df['item'].unique()]
stats_df = pd.DataFrame(stats_list)

scaler = RobustScaler()
X_scaled = scaler.fit_transform(stats_df[['trend_strength', 'seasonality_strength']])

kmeans = KMeans(n_clusters=6, random_state=42)
stats_df['cluster'] = kmeans.fit_predict(X_scaled)

stats_df = stats_df[['item', 'cluster']]
stats_df[['trend_scaled', 'seasonality_scaled']] = X_scaled

In [56]:
stats_df

,item,cluster,trend_scaled,seasonality_scaled
0,আটা (প্যাকেট),0,-0.497942,-0.403584
1,আটা সাদা (খোলা),0,-0.525549,-0.393064
2,আদা (আমদানি),0,0.331129,0.677489
3,আদা (দেশী),0,1.764233,0.930259
4,আলু (মানভেদে),0,-0.388650,-0.287071
5,ইলিশ,4,5.587418,15.890082
6,"এম,এস রড (৬০ গ্রেড)",2,124.873066,59.381514
7,"এম,এস রড( ৪০ গ্রেড)",1,140.043756,2156.823630
8,এলাচ(ছোট),5,13.564054,6.622221
9,এ্যাংকর ডাল,0,-0.434376,-0.399732


In [58]:
featured_df = df.merge(stats_df[['item', 'cluster', 'trend_scaled', 'seasonality_scaled']], on='item', how='left')

featured_df['date_ordinal'] = featured_df['date'].map(pd.Timestamp.toordinal)
X = featured_df[['date_ordinal', 'cluster', 'trend_scaled', 'seasonality_scaled']]
y = featured_df['high_price']

model_with_features = RandomForestRegressor(n_estimators=100, random_state=42)
model_with_features.fit(X, y)
predictions_with = model_with_features.predict(X)

model_baseline = RandomForestRegressor(n_estimators=100, random_state=42)
model_baseline.fit(X[['date_ordinal']], y)
predictions_baseline = model_baseline.predict(X[['date_ordinal']])

mse_with = mean_squared_error(y, predictions_with)
mse_baseline = mean_squared_error(y, predictions_baseline)

print(f"MSE with new features: {mse_with:.2f}")
print(f"MSE baseline: {mse_baseline:.2f}")
print(f"Improvement: {((mse_baseline - mse_with) / mse_baseline * 100):.1f}%")

importances = model_with_features.feature_importances_
feature_names = X.columns
for name, importance in zip(feature_names, importances):
    print(f"{name}: {importance:.3f}")

MSE with new features: 1117879.98
MSE baseline: 297429301.46
Improvement: 99.6%
date_ordinal: 0.025
cluster: 0.004
trend_scaled: 0.474
seasonality_scaled: 0.497
